In [1]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

Using TensorFlow backend.


In [2]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [3]:
def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    encoder, decoder = models
    x_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test, batch_size=batch_size)
    
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1])#, c=y_test)
    # plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    # plt.savefig(filename)
    plt.show()

In [4]:
# MNIST dataset
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print("raw xtrain:", x_train.shape)
# print("raw ytrain:", y_train.shape)
# print("raw xtest:", x_test.shape)
# print("raw ytest:", y_test.shape)

# image_size = x_train.shape[1]
# original_dim = image_size * image_size
# x_train = np.reshape(x_train, [-1, original_dim]).astype('float32') / 255
# x_test = np.reshape(x_test, [-1, original_dim]).astype('float32') / 255

x_l_data = np.loadtxt("../../dataset/diag_labrococo.txt")

In [5]:
split_data_at = .9
print(x_l_data.shape[0]*split_data_at)
x_train = x_l_data[:int(x_l_data.shape[0]*split_data_at),:]
x_test = x_l_data[int(x_l_data.shape[0]*split_data_at):,:]
original_dim = x_l_data.shape[1]

print("x_l_data:", x_l_data.shape)
print("xtrain:", x_train.shape)
print("xtest:", x_test.shape)

2514.6
x_l_data: (2794, 728)
xtrain: (2514, 728)
xtest: (280, 728)


In [6]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
#plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
#plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 728)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          373248      encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            1026        dense_1[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 2)            1026        dense_1[0][0]                    
__________________________________________________________________________________________________
z (Lambda)

In [7]:
models = (encoder, decoder)
data = x_test
reconstruction_loss = binary_crossentropy(inputs, outputs)

In [8]:
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()
# plot_model(vae, to_file='vae_mlp.png', show_shapes=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 728)               0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 375300    
_________________________________________________________________
decoder (Model)              (None, 728)               375000    
Total params: 750,300
Trainable params: 750,300
Non-trainable params: 0
_________________________________________________________________


In [10]:
epochs = 10

vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))

Train on 2514 samples, validate on 280 samples
Epoch 1/10
2514/2514 [==============================] - 1s 319us/step - loss: -5933.0809 - val_loss: -17155.3390
Epoch 2/10
2514/2514 [==============================] - 0s 120us/step - loss: -15013.8087 - val_loss: -19266.6774
Epoch 3/10
2514/2514 [==============================] - 0s 135us/step - loss: -15926.8149 - val_loss: -19810.5252
Epoch 4/10
2514/2514 [==============================] - 0s 125us/step - loss: -16155.7439 - val_loss: -19988.8446
Epoch 5/10
2514/2514 [==============================] - 0s 120us/step - loss: -16232.2529 - val_loss: -20027.7458
Epoch 6/10
2514/2514 [==============================] - 0s 131us/step - loss: -15794.3035 - val_loss: -20075.9662
Epoch 7/10
2514/2514 [==============================] - 0s 142us/step - loss: -15539.3410 - val_loss: -19662.9995
Epoch 8/10
2514/2514 [==============================] - 0s 134us/step - loss: -16125.1940 - val_loss: -19863.0943
Epoch 9/10
2514/2514 [====================

In [ ]:
plot_results(models,
             data,
             batch_size=batch_size,
             model_name="vae_mlp")